In [1]:
from utils.logger import Tracer, print_header
from models import Client, Server
from models.message import AAD
from datetime import datetime, date
import questionary
import sys

## Logging/pwd change

In [2]:
if True:
    server = Server(name = 'Server')
    transmitter = Client(name = 'Transmitter', password = 'password@transmitter')
    transmitter.register_on(server)
    transmitter.login_on(server)
    transmitter.change_password('myNew@password')
    transmitter.logout()

[DEBUG]: [Transmitter]: Drawing a random salt...
[DEBUG]: [Transmitter]: Hashing password...
[DEBUG]: [Transmitter]: Request a registration on Server
[INFO]: [Server]: New user Transmitter was added!
[DEBUG]: [Transmitter]: Getting salt from Server
[DEBUG]: [Transmitter]: Recomputing pwd_verifier
[DEBUG]: [Transmitter]: Sending login request to Server
[INFO]: [Server]: User Transmitter is now connected!
[DEBUG]: [Transmitter]: Drawing a random salt...
[DEBUG]: [Transmitter]: Hashing password...
[DEBUG]: [Transmitter]: Request a registration on Server
[INFO]: [Server]: Password updated for Transmitter.
[DEBUG]: [Transmitter]: Sending logout request to Server
[INFO]: [Server]: Transmitter has been logged out.


## Messages

In [4]:
server = Server(name = 'Server')
alice = Client(name = 'Alice', password = 'alicepwd')
bob = Client(name = 'Bob', password = 'bobpwd')

alice.register_on(server)
bob.register_on(server)
print('--')

[DEBUG]: [Alice]: Drawing a random salt...
[DEBUG]: [Alice]: Hashing password...
[DEBUG]: [Alice]: Request a registration on Server
[INFO]: [Server]: New user Alice was added!
[DEBUG]: [Bob]: Drawing a random salt...
[DEBUG]: [Bob]: Hashing password...
[DEBUG]: [Bob]: Request a registration on Server
[INFO]: [Server]: New user Bob was added!
--


In [5]:
alice.login_on(server)

alice.send_message(data='Hi Bob!', recipient_name='Bob', unlock_day=date(year=2025, month=5, day=30))
alice.send_message(data='You will be able to read this message later!', recipient_name='Bob', unlock_day=date(year=2025, month=6, day=8))

[DEBUG]: [Alice]: Getting salt from Server
[DEBUG]: [Alice]: Recomputing pwd_verifier
[DEBUG]: [Alice]: Sending login request to Server
[INFO]: [Server]: User Alice is now connected!
[DEBUG]: [Alice]: Getting Bob public key on Server
[DEBUG]: [Alice]: Generating a public key
[DEBUG]: [Alice]: Encrypting message data
[DEBUG]: [Alice]: Encrypting symmetric key
[DEBUG]: [Alice]: Sending message on Server
[INFO]: [Server]: Message sent to Bob.
[DEBUG]: [Alice]: Getting Bob public key on Server
[DEBUG]: [Alice]: Generating a public key
[DEBUG]: [Alice]: Encrypting message data
[DEBUG]: [Alice]: Encrypting symmetric key
[DEBUG]: [Alice]: Sending message on Server
[INFO]: [Server]: Message sent to Bob.


True

In [9]:
bob.login_on(server)
aad = bob.get_my_messages()
print(aad[0])

print('Received message :', bob.read_message(0))
print('Key of message 0 :', bob.get_message_key(0))

print('Received message :', bob.read_message(1))
print('Downloaded message :', bob.download_future_message(1))
print('Received key from download :', bob.download_future_message(1).key)

[DEBUG]: [Bob]: Getting salt from Server
[DEBUG]: [Bob]: Recomputing pwd_verifier
[DEBUG]: [Bob]: Sending login request to Server
[INFO]: [Server]: User Bob is now connected!
[DEBUG]: [Bob]: Requesting message metadata from Server
Message: From: Alice | To: Bob | Unlock day:2025-05-30
[DEBUG]: [Bob]: Requesting full message (id:0) from Server
[INFO]: [Server]: Returning message (id:0) with key
[DEBUG]: [Bob]: Decrypting message content
Received message : Hi Bob!
[DEBUG]: [Bob]: Requesting key for message (id:0)
Key of message 0 : b"R\xd1kl\xbc\x8bs^\x8dk'\xd3\xc2V@C\x11\xe67\x89\x1d\xa6\xb5O:\xcf\xdc\xe7[\x08\xb5T"
[DEBUG]: [Bob]: Requesting full message (id:1) from Server
[WARNING]: [Server]: Access to message (id:1) is restricted until 2025-06-08
[ERROR]: [Bob]: Unable to read message (id:1)
Received message : None
[DEBUG]: [Bob]: Downloading future message (id:1) without key
[INFO]: [Server]: Returning future message (id:1) without key
Downloaded message : Message: From: Alice | To: